## Matrix Multiplication from Scratch

In [2]:
import torch
from torch import tensor

In [3]:
def matmul(a, b):
    ar,ac = a.shape 
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i, k] * b[k, j]
    
    return c

In [4]:
# lets test the speed of this function versus pytorch's built-in method
m1 = torch.randn(5, 28*28)
m2 = torch.rand(784, 10)

In [5]:
%time t1=matmul(m1, m2)

CPU times: user 1.48 s, sys: 6.07 ms, total: 1.49 s
Wall time: 1.53 s


In [6]:
%time t2=m1@m2

CPU times: user 2.13 ms, sys: 6.69 ms, total: 8.83 ms
Wall time: 41 ms


Pytorch is about 4 orders of magnitude faster due to functions being optimized and written in a lower level language that is C++

In [7]:
# we can eleminate eleminate the last for loop in favor of pytorch elementwise arithmatic and sum function 
def matmul(a, b):
    ar,ac = a.shape 
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i, j] = (a[i] * b[:,j]).sum()
    return c

%time t3=matmul(m1,m2)

CPU times: user 1.17 ms, sys: 2.85 ms, total: 4.02 ms
Wall time: 13.9 ms


In [8]:
# make it even faster with broadcasting and eleminate another inner loop
def matmul(a, b):
    ar,ac = a.shape 
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
            c[i] = (a[i].unsqueeze(-1) * b).sum(dim=0)
    
    return c

%time t4=matmul(m1,m2)

CPU times: user 70 µs, sys: 836 µs, total: 906 µs
Wall time: 3.18 ms


In [9]:
# use einstein summation
def matmul(a,b): return torch.einsum('ik,kj->ij', a, b)

%time t5=matmul(m1, m2)

CPU times: user 772 µs, sys: 67 µs, total: 839 µs
Wall time: 3.21 ms


## Forward Pass of Neural Nets

The neural net layer will be 1 linear layer followed by a ReLU activation, and then another linear layer. 

We will have 100 inputs and deal with a batch size of 200.

In [10]:
# random inputs and targets
x = torch.randn(200, 100)
y = torch.rand(200)

# initialize the weights properly using the Kaimi He initialization
from math import sqrt 
w1 = torch.randn(100,50) / sqrt(2/100)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1) / sqrt(2/50)
b2  = torch.zeros(1)


def lin(x, w, b): return (x @ w) + b
def relu(x): return x.clamp_min(0.)

def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

out = model(x)
print(out.shape)
print(out[0:10])

torch.Size([200, 1])
tensor([[  362.1924],
        [ 1227.9335],
        [-1271.0524],
        [  975.6238],
        [-1367.2017],
        [  269.5671],
        [-1786.2316],
        [   20.4437],
        [   -3.3740],
        [ -755.9406]])


In [11]:
# loss function
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

loss = mse(out, y)
loss

tensor(1671133.1250)

## Backward Pass

Using the chain rule. 

In [12]:
def mse_grad(input, targ):
    '''
    Go back one layer and calculate its gradient wrt the loss
    Use the fact d(x^2)/x = 2x  
    '''
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/ inp.shape[0]

In [13]:
def relu_grad(inp, out):
    '''
    derivative of relu is 1 is inp > 0, and 0 otherwise.
    '''
    inp.g = (input>0).float() * out.g

In [14]:
'''
Using matrix multiplation, the chain rule, and the multivariate case of the chain rule, 
the following end up being the expressions for linear transformation gradients during back prop
'''
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

## Refractoring

In [15]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [16]:
class Lin():
    
    def __init__(self, w, b): self.w, self.b = w,b
    
    def __call__(self, inp): 
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)
    

In [17]:
class Mse():
    
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        x = (self.inp.squeeze(-1) - self.targ).unsqueeze(-1)
        self.inp.g = 2. * x /self.targ.shape[0]

In [18]:
class Model(): 
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [19]:
model = Model(w1, b1, w2, b2)

In [20]:
#forward pass
loss = model(x,y)
loss

tensor(1671133.1250)

In [21]:
#backward pass
model.backward()
model.layers[0].w.shape, model.layers[0].w.g

(torch.Size([100, 50]),
 tensor([[  24.4630,   87.9614, -370.8536,  ..., -334.2465,  100.3873,
           751.4680],
         [ -42.6653,  137.1520, -102.6144,  ...,  866.1729,   81.7429,
           199.9442],
         [ -16.2776, -187.6119,   81.0970,  ...,  557.0064, -208.2754,
          -882.6351],
         ...,
         [ 144.0176, -564.4271, -301.1157,  ..., -280.9814,  279.4561,
          1158.8986],
         [-156.2803, 1278.6521,  440.5824,  ..., 1080.0415, -166.5890,
          -125.3314],
         [ -32.9319, -343.3602,  117.2498,  ..., 1228.2921,   95.9142,
           230.6053]]))

In [63]:
# implement the module in pytorch
import torch.nn as nn

def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

class MyModel(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out) # Uses the Kaimi He Initialization
        )
        self.loss = mse # function defined before the class
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)

In [64]:
model1 = MyModel(100, 50, 1)

In [69]:
x=torch.randn(200, 100); y = torch.randn(200)
loss = model1.forward(x, y)
loss.item()

1.1328355073928833

In [76]:
params = model1.parameters()
for param in params:
    print(param)

Parameter containing:
tensor([[ 0.0819,  0.0818, -0.0708,  ...,  0.0116, -0.0231,  0.0521],
        [ 0.0647, -0.0037, -0.0948,  ...,  0.0344,  0.0546, -0.0359],
        [-0.0960,  0.0063, -0.0359,  ...,  0.0216, -0.0785, -0.0452],
        ...,
        [-0.0142,  0.0176, -0.0441,  ...,  0.0563,  0.0569, -0.0083],
        [-0.0782, -0.0801,  0.0844,  ..., -0.0517, -0.0979,  0.0019],
        [-0.0319, -0.0254, -0.0564,  ..., -0.0618,  0.0834,  0.0028]],
       requires_grad=True)
Parameter containing:
tensor([-0.0223,  0.0071, -0.0971, -0.0787, -0.0889, -0.0133, -0.0433,  0.0085,
         0.0457,  0.0402, -0.0025, -0.0380, -0.0826,  0.0225, -0.0827,  0.0718,
         0.0301, -0.0991, -0.0261, -0.0869, -0.0575,  0.0776, -0.0606, -0.0086,
        -0.0924,  0.0310,  0.0946,  0.0300, -0.0952,  0.0028,  0.0395, -0.0371,
         0.0603, -0.0446, -0.0639,  0.0399, -0.0084,  0.0335, -0.0152, -0.0730,
         0.0597, -0.0118, -0.0444, -0.0154, -0.0476,  0.0344,  0.0029,  0.0884,
        -0.0777